In [74]:
import pandas as pd
import numpy as np

file_to_load = "Resources/Datafiniti_Fast_Food_Restaurants.csv"
file_to_load2 = "Resources/crime_data_w_population_and_crime_rate.csv"


fastfood_data = pd.read_csv(file_to_load)
ff_counts = fastfood_data['province'].value_counts()
ff_bystate = pd.DataFrame(ff_counts)
ff_bystate = ff_bystate.reset_index(inplace=False)
ff_bystate = ff_bystate.rename(columns={"index":"state"})
ff_bystate = ff_bystate.rename(columns={"province":"number"})
ff_bystate['number'] = ff_bystate['number'].astype(float)
ff_bystate['state'] = ff_bystate['number'].astype(float)

CA    1201
TX     811
FL     621
OH     522
GA     420
IL     405
PA     383
MI     374
NY     352
AZ     330
TN     302
NC     295
IN     254
VA     253
MA     205
LA     202
MN     199
WA     196
WI     189
SC     188
MD     172
OK     166
KY     166
MO     163
OR     154
CO     148
NJ     129
NV     121
IA     115
AR     102
NE      87
NM      78
KS      74
WV      70
UT      63
MS      55
CT      53
ID      51
DE      44
SD      42
MT      38
ND      35
NH      34
HI      32
WY      29
ME      25
AK      16
RI      15
VT      15
AL       6
Name: province, dtype: int64


In [70]:
crime_data = pd.read_csv(file_to_load2)
crime_data[['county','state']] = crime_data['county_name'].str.split(',',expand=True)
crime_data["crime_rate"] = crime_data['crime_rate_per_100000']/1000
crime_data = crime_data[['state','crime_rate']]
crime_bystate = crime_data.groupby(['state'])
avg_bystate = crime_bystate.mean()
avg_bystate["crime_rate"] = round(avg_bystate["crime_rate"],2)
avg_bystate = avg_bystate.drop(avg_bystate.index[7])
avg_bystate = avg_bystate.reset_index(inplace=False)
avg_bystate



,state,crime_rate
0,AK,0.29
1,AL,0.39
2,AR,0.33
3,AZ,0.35
4,CA,0.42
5,CO,0.18
6,CT,0.18
7,DE,0.52
8,FL,0.41
9,GA,0.30
